In [ ]:
import cv2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

# Initialize the camera
camera = cv2.VideoCapture(0)  # 0 represents the default camera, change it if necessary

# Load a pre-trained face detection model (you may need to install OpenCV's cascades)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Email configuration
sender_email = "your_email@gmail.com"
sender_password = "your_password"
receiver_email = "receiver_email@gmail.com"

# Function to send an email with the detected intruder's image
def send_email(image_path):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = 'INTRUDER ALERT'

    text = MIMEText("An intruder was detected.")
    msg.attach(text)

    with open(image_path, 'rb') as img_file:
        image = MIMEImage(img_file.read())
        msg.attach(image)

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()

while True:
    ret, frame = camera.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Save the detected face as an image
        face_image = frame[y:y + h, x:x + w]
        cv2.imwrite("intruder_face.jpg", face_image)

        # Send an email with the intruder's image
        send_email("intruder_face.jpg")

    # Display the frame
    cv2.imshow('Intruder Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
camera.release()
cv2.destroyAllWindows()
